# SetUp

In [1]:
import os
import sys
import glob
from os.path import abspath
os.environ['SPARK_HOME'] = '/home/chiara/Documenti/BigData/CountingTriangles/spark-3.5.0-bin-hadoop3'
os.environ['HADOOP_HOME'] = '/home/chiara/Documenti/BigData/CountingTriangles/spark-3.5.0-bin-hadoop3'
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-17-openjdk-amd64'
os.environ['SPARK_LOCAL_IP'] = '172.17.0.1'

In [2]:
spark_python = os.path.join(os.environ.get('SPARK_HOME',None),'python')
py4j = glob.glob(os.path.join(spark_python,'lib','py4j-*.zip'))[0]
graphf = glob.glob(os.path.join(spark_python,'graphframes.zip'))[0]
sys.path[:0]=[spark_python,py4j]
sys.path[:0]=[spark_python,graphf]
os.environ['PYTHONPATH']=py4j+os.pathsep+graphf

In [3]:
import findspark
findspark.init()
findspark.find()

'/home/chiara/Documenti/BigData/CountingTriangles/spark-3.5.0-bin-hadoop3'

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Counting Triangles").enableHiveSupport().getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/21 18:32:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
from graphframes import *
import networkx as nx
import matplotlib.pyplot as plt

# Parsing Dataset

In [6]:
with open('dataset/Wiki-Vote.txt','r') as f:
    content = f.readlines()

edges_list = list(filter( lambda x: not x.startswith('#') ,content))
edges = list(map(lambda x: tuple(x.split('\t')), edges_list))
edges_tuples = list(map(lambda x: (int(x[0]), int(x[1].replace('\n',''))), edges))

In [7]:
from pyspark.sql.types import IntegerType, Row

list1, list2 = zip(*edges_tuples)
nodes = list(set(list1 + list2))
nodes_tuple = [Row(x) for x in nodes]

In [8]:
from pyspark.sql.functions import col, collect_list

#get list of nodes, with columns renamed value and id
vertices = spark.createDataFrame(nodes, IntegerType())
vertices = vertices.withColumnRenamed('value','id')

#get edges such that the src node is always smaller then the dst node
edges_n = spark.createDataFrame(edges_tuples,["src", "dst"],IntegerType())
edges_inverted = edges_n.filter(edges_n.src > edges_n.dst)
edges_normal = edges_n.filter(edges_n.src < edges_n.dst)
edges_normal2 = edges_inverted.select(col('dst').alias('src'),col('src').alias('dst'))
edges = edges_normal.union(edges_normal2).distinct()

# Query Implementation

In [9]:
e1 = edges.alias('e1')
e2 = edges.alias('e2')
e3 = edges.alias('e3')

In [10]:
e1.show()

+---+----+
|src| dst|
+---+----+
|  3| 586|
| 25| 255|
| 25| 590|
|  7|1193|
|  8| 232|
|  8| 607|
| 11| 958|
| 11|1437|
| 11|2595|
| 19|  61|
| 23| 302|
| 47|3352|
| 29|3717|
| 29|4088|
| 86|1305|
| 99|5812|
|103|2871|
|108|8283|
|109|7052|
|122|3258|
+---+----+
only showing top 20 rows



In [12]:
from pyspark.sql.functions import col

result = e1.join(e2, col("e1.src") == col("e2.src")) \
    .join(e3, (col("e1.dst") == col("e3.src")) & (col("e2.dst") == col("e3.dst"))) \
    .select(col("e1.src").alias("node1"), col("e1.dst").alias("node2"), col("e2.dst").alias("node3")).distinct()

In [13]:
result.count()

608389

In [110]:
result.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[node1#87L, node2#88L, node3#89L], functions=[])
   +- HashAggregate(keys=[node1#87L, node2#88L, node3#89L], functions=[])
      +- Project [src#4L AS node1#87L, dst#5L AS node2#88L, dst#60L AS node3#89L]
         +- SortMergeJoin [dst#5L, dst#60L], [src#71L, dst#72L], Inner
            :- Sort [dst#5L ASC NULLS FIRST, dst#60L ASC NULLS FIRST], false, 0
            :  +- Exchange hashpartitioning(dst#5L, dst#60L, 200), ENSURE_REQUIREMENTS, [plan_id=11647]
            :     +- Project [src#4L, dst#5L, dst#60L]
            :        +- SortMergeJoin [src#4L], [src#59L], Inner
            :           :- Sort [src#4L ASC NULLS FIRST], false, 0
            :           :  +- Exchange hashpartitioning(src#4L, 200), ENSURE_REQUIREMENTS, [plan_id=11637]
            :           :     +- HashAggregate(keys=[src#4L, dst#5L], functions=[])
            :           :        +- Exchange hashpartitioning(src#4L, dst#5L, 200), 

# Confront with another implementation

In [14]:
graph = GraphFrame(vertices,edges)

In [15]:
triangles = graph.triangleCount()

In [16]:
triangles.show()

+-----+---+
|count| id|
+-----+---+
|  197| 12|
|    0| 22|
|   12| 13|
| 3143|  6|
|   23| 16|
|  280|  3|
| 1078| 20|
|   88|  5|
|  482| 19|
| 4847| 15|
|  321|  9|
|  156| 17|
|   95|  4|
| 2294|  8|
|  661| 23|
|   42|  7|
|  700| 10|
|  164| 21|
|13401| 11|
|  361| 14|
+-----+---+
only showing top 20 rows



In [33]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import sum

triangle_count = triangles.select(sum("count")/3)
triangle_count.select(col('(sum(count) / 3)').cast(IntegerType()).alias('count')).show()

+------+
| count|
+------+
|608389|
+------+



# Confront with another algorithm

In [65]:
def is_good_line(line):
    # filter lines in the input file
    try:
        fields = line.split('\t')
        if len(fields) > 2:
            return False

        float(fields[-1])
        return True
    except:
        return False

In [56]:
import itertools

def generate_combinations(line):
    # create all combinations and return key-value pairs in format ((v1,v2), ['to_check'])
    l = line[1]
    return [(pair, ['to_check']) for pair in itertools.combinations(l, 2)]

In [57]:
def degree_filter(t):
    # not considering vertices with degree less than 2
    # as there are no possible wedges/closed wedges which they could form
    if len(t[1]) < 2:
        return False
    return True

In [66]:
sc = spark.sparkContext
lines = sc.textFile("./dataset/Wiki-Vote.txt")
rdd_edges = lines.filter(is_good_line)

In [81]:
# will check for edges only, so create key-value pairs ((v1, v2), 'exists')
existing_edges_1 = rdd_edges.map(lambda l: ((l.split('\t')[0], l.split('\t')[1]), ['exists']))
existing_edges_2 = rdd_edges.map(lambda l: ((l.split('\t')[1], l.split('\t')[0]), ['exists']))
existing_edges = existing_edges_1.union(existing_edges_2)

In [88]:
existing_edges.take(5)

[(('30', '1412'), ['exists']),
 (('30', '3352'), ['exists']),
 (('30', '5254'), ['exists']),
 (('30', '5543'), ['exists']),
 (('30', '7478'), ['exists'])]

In [89]:
edges_1 = rdd_edges.map(lambda l: (l.split('\t')[0], l.split('\t')[1]))
edges_2 = rdd_edges.map(lambda l: (l.split('\t')[0], l.split('\t')[1]))

edges = edges_1.union(edges_2)

In [91]:
# create rdd that has key-value pairs
# (vertex1, [nb1,nb2,nb3...]), where nbi is neighbour i of vertex1
neighbours = edges.groupByKey().map(lambda x : (x[0], list(set(list(x[1])))))
neighbours = neighbours.filter(degree_filter)

In [92]:
# generate all pairs ((v1, v2), ['to_check'])
# use flatMap as generate_combinations returns lists
edges_to_check = neighbours.map(generate_combinations).flatMap(lambda x: x)

In [93]:
# perform summation over lists containing 'to_check'
edges_to_check = edges_to_check.reduceByKey(lambda a,b: a+b)

In [95]:
# create rdd in the format ((v1, v2), ('exists', ['to_check', 'to_check',...]))
all_edges = existing_edges.join(edges_to_check)
all_edges = all_edges.map(lambda x: (x[0], len(x[1][1])))

In [96]:
# separate count for each vertex
vertices_part1 = all_edges.map(lambda y: (y[0][0], y[1]))
vertices_part2 = all_edges.map(lambda y: (y[0][1], y[1]))
vertex_pair = vertices_part1.union(vertices_part2)

In [107]:
# summing over all vertex counts
vertex = vertex_pair.reduceByKey(lambda a,b: a+b)
final = vertex.map(lambda x: (x[0], x[1]/6))

In [108]:
from operator import add
triangle_count2 = final.map(lambda x: x[1]).reduce(add)

In [109]:
triangle_count2

248852.33333333337